***Week 5 Assignment: Choosing best possible location for an Asian restaurant in neighborhoods of Toronto***

In [1]:
import requests
import pandas as pd
import numpy as np

***Initial steps will be same as the notebook in the lab because getting the postal code of the neighborhoods of Toronto and other steps like storing them in a data frame and then analysing it willbe same.***

In [2]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [3]:
df = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#Drop the the rows on which the Borough is empty
df.dropna(subset=['Borough'], inplace=True)

In [5]:
#Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df[df['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


***So, there is no row with an empty neighborhood. We can now proceed for our further step which is grouping based on postal codes or Borough***

In [6]:
df_postcodes = df.groupby(['Postal Code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)])
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_postcodes.shape

(103, 3)

***Reading the Geospatial coordinates csv file.***

In [8]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


***This data frame and the previous pre processed data frame for postal codes has the "Postal Codes" group as common. Merging the two.***

In [9]:
# Make sure both dataframes have the same 
df_data.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_postcodes.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [10]:
# Merge both datasets
df_neighborhoods = pd.merge(df_postcodes, df_data, on='PostalCode')
df_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


***Since, we have refined the grouped data we can now move to map plotting and clustering in order to fulfil our potential.***

In [11]:
!pip -q install folium
import folium
from sklearn.cluster import KMeans

In [12]:
print('The dataframe has: \nBoroughs:{}\nNeighborhoods:{} '.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has: 
Boroughs:10
Neighborhoods:103 


In [13]:
#count Bourough and Neighborhood
df_neighborhoods.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [14]:
df_toronto = df_neighborhoods[df_neighborhoods['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [15]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [16]:
#Creating a list of all the Boroughs
boroughs = df_toronto['Borough'].unique().tolist()

In [17]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [18]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

In [20]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# Using four square api's

In [25]:
# define Foursquare Credentials and Version
CLIENT_ID = 'LXB3FXJS255CWZUBAMGJQVSDTZR3FUZ3RW3BBSUGYK0IRW3V' # your Foursquare ID
CLIENT_SECRET = '12DGKE2M5EJGLLDZDW11GMGVSQU34XCQ2B0PFZLNRSBG44HC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Your credentails:
CLIENT_ID: LXB3FXJS255CWZUBAMGJQVSDTZR3FUZ3RW3BBSUGYK0IRW3V
CLIENT_SECRET:12DGKE2M5EJGLLDZDW11GMGVSQU34XCQ2B0PFZLNRSBG44HC


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [28]:
#Check size of resulting dataframe
toronto_venues.shape

(1611, 7)

In [29]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [30]:
# Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [31]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


In [32]:
#print out the list of categories
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Restaurant',
       'Pizza Place', 'Juice Bar', 'Bookstore', 'Bubble Tea Shop',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Coffee Shop',
       'Bakery', 'Caribbean Restaurant', 'Café', 'Indian Restaurant',
       'Japanese Restaurant', 'Lounge', 'Frozen Yogurt Shop',
       'American Restaurant', 'Liquor Store', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Park', 'Pet Store',
       'Steakhouse', 'Burrito Place', 'Movie Theater', 'Sandwich Place',
       'Food & Drink Shop', 'Fish Market', 'Gay Bar',
       'Seafood Restaurant', 'Cheese Shop', 'Middle Eastern Restaurant',
       'Comfort Food Restaurant', 'Stationery Store', 'Wine Bar',
       'Thai Restaurant', 'Coworking Space', 'Latin American Restaurant',
 

In [33]:
# check if the results contain "Chinese Restaurants"
"Chinese Restaurant" in toronto_venues['Venue Category'].unique()

True

***Analyzing neighborhoods.***

In [34]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1611, 237)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 237)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.000000,0.015625
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.025316
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
len(to_grouped[to_grouped["Chinese Restaurant"] > 0])

7

In [37]:
to_asian = to_grouped[["Neighborhoods","Thai Restaurant"]]

In [38]:
to_asian.head()

,Neighborhoods,Thai Restaurant
0,Berczy Park,0.018182
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.015625


In [39]:
to_chinese = to_grouped[["Neighborhoods","Chinese Restaurant"]]

In [40]:
to_chinese.head()

,Neighborhoods,Chinese Restaurant
0,Berczy Park,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.015625


# Clustering Toronto's neighborhoods

In [41]:
# set number of clusters
toclusters = 3

to_clustering = to_chinese.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 0, 0, 0, 0, 0], dtype=int32)

In [42]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_chinese.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [43]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

,Neighborhood,Chinese Restaurant,Cluster Labels
0,Berczy Park,0.000000,0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0
2,"Business reply mail Processing Centre, South C...",0.000000,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
4,Central Bay Street,0.015625,2


In [44]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1611, 9)


,Neighborhood,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,0,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [45]:
# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1611, 9)


,Neighborhood,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.000000,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,The Sweet Escape Patisserie,43.650632,-79.358709,Bakery
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,ODIN Cafe + Bar,43.656739,-79.356503,Café
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,John Fluevog Shoes,43.649896,-79.359436,Shoe Store
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,Soulpepper Theatre,43.650780,-79.357615,Theater
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,Greg's Ice Cream,43.650102,-79.360440,Ice Cream Shop
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,Berkeley Church,43.655123,-79.365873,Event Space
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,Alumnae Theatre,43.652756,-79.364753,Theater
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,Caffe Furbo,43.649970,-79.358849,Café
24,"Regent Park, Harbourfront",0.000000,0,43.654260,-79.360636,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


In [46]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [47]:
# create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining Clusters

In [81]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,The Sweet Escape Patisserie,43.650632,-79.358709,Bakery
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,ODIN Cafe + Bar,43.656739,-79.356503,Café
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,John Fluevog Shoes,43.649896,-79.359436,Shoe Store
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Soulpepper Theatre,43.650780,-79.357615,Theater
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Greg's Ice Cream,43.650102,-79.360440,Ice Cream Shop
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Berkeley Church,43.655123,-79.365873,Event Space
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Alumnae Theatre,43.652756,-79.364753,Theater
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,Caffe Furbo,43.649970,-79.358849,Café
24,"Regent Park, Harbourfront",0.0,0,43.654260,-79.360636,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


In [82]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
38,"University of Toronto, Harbord",0.029412,1,43.662696,-79.400049,Almond Butterfly,43.662836,-79.403365,Bakery
38,"University of Toronto, Harbord",0.029412,1,43.662696,-79.400049,Cafe Cancan,43.662735,-79.403447,French Restaurant
38,"University of Toronto, Harbord",0.029412,1,43.662696,-79.400049,Bakka Phoenix Books,43.662959,-79.402601,Bookstore
38,"University of Toronto, Harbord",0.029412,1,43.662696,-79.400049,Harbord House,43.662466,-79.405410,Bar
30,"St. James Town, Cabbagetown",0.043478,1,43.667967,-79.367675,Toronto Dance Theatre,43.666232,-79.367075,General Entertainment
30,"St. James Town, Cabbagetown",0.043478,1,43.667967,-79.367675,St. Jamestown Delicatessen,43.665811,-79.368648,Butcher
30,"St. James Town, Cabbagetown",0.043478,1,43.667967,-79.367675,Pet Valu,43.664205,-79.368460,Pet Store
30,"St. James Town, Cabbagetown",0.043478,1,43.667967,-79.367675,Stout Irish Pub,43.663891,-79.369030,Pub
30,"St. James Town, Cabbagetown",0.043478,1,43.667967,-79.367675,House on Parliament,43.663646,-79.367854,Gastropub
38,"University of Toronto, Harbord",0.029412,1,43.662696,-79.400049,Magic Noodle,43.662728,-79.403602,Noodle House


In [83]:
#Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Chinese Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,Chotto Matte,43.646473,-79.378782,Japanese Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,Sam James Coffee Bar (SJCB),43.647881,-79.384332,Café
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,Walrus Pub & Beer Hall,43.647375,-79.379515,Pub
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,Pilot Coffee Roasters,43.645018,-79.380415,Coffee Shop
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,Pumpernickel's Deli,43.648832,-79.381970,Deli / Bodega
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,The Fairmont Royal York,43.645449,-79.381508,Hotel
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,Maman,43.648309,-79.382253,Café
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,DAVIDsTEA,43.646506,-79.380145,Tea Room
37,"Toronto Dominion Centre, Design Exchange",0.010000,2,43.647177,-79.381576,Mos Mos Coffee,43.648159,-79.378745,Café


# Analyzed points:

***->*** Most of the Chinese restaurants is in cluster 2 followed by cluster 3.

***->*** So, the business can start off by competing with chinese restaurants in cluster 3 and then go on exploring in the cluster 1.

***->*** Another strategy would be starting off the restaurant in cluster 1 and create a new marketfor itself.